Сделать свою гибридную рекомендательную систему

In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

C:\Users\koloskov\AppData\Local\Temp\ipykernel_10940\49583723.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


TF-IDFстатистическая мера,
используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.

In [24]:
import pandas as pd

In [68]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [57]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [59]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [70]:
movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [71]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)
X_train_tfidf

<9742x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [72]:
pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [115]:
neigh = NearestNeighbors(n_neighbors=100, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_neighbors=100)

In [116]:
film = movies[movies.title == 'Fight Club (1999)']
film 

,movieId,title,genres
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller


In [117]:
# lets get recomndation on these genres
test = change_string("Action|Crime|Drama|Thriller")
X_tfidf = tfidf.transform([test])
res = neigh.kneighbors(X_tfidf, return_distance=True)

In [118]:
res

(array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.35312756, 0.35312756, 0.35312756, 0.35312756,
         0.35312756, 0.35312756, 0.35312756, 0.35312756, 0.35312756,
         0.35312756, 0.35312756, 0

In [119]:
movies.iloc[res[1][0]] # this is what first recomendation system recomend based on KNN recomend for film "fight club'

,movieId,title,genres
2165,2881,Double Jeopardy (1999),Action|Crime|Drama|Thriller
7236,73741,Ninja (2009),Action|Crime|Drama|Thriller
2443,3256,Patriot Games (1992),Action|Crime|Drama|Thriller
8797,130083,Kidnapping Mr. Heineken (2015),Action|Crime|Drama|Thriller
9503,170875,The Fate of the Furious (2017),Action|Crime|Drama|Thriller
...,...,...,...
3036,4064,Coffy (1973),Action|Crime|Thriller
2802,3744,Shaft (2000),Action|Crime|Thriller
2062,2741,No Mercy (1986),Action|Crime|Thriller
1315,1769,"Replacement Killers, The (1998)",Action|Crime|Thriller


In [120]:
movies_rec_syst_1 = list(movies.iloc[res[1][0]].title)
movies_rec_syst_1

['Double Jeopardy (1999)',
 'Ninja (2009)',
 'Patriot Games (1992)',
 'Kidnapping Mr. Heineken (2015)',
 'The Fate of the Furious (2017)',
 'I Am Wrath (2016)',
 'Run All Night (2015)',
 'Munich (2005)',
 'Death Wish (2018)',
 'Baader Meinhof Komplex, Der (2008)',
 'Contraband (2012)',
 'Shot Caller (2017)',
 'Real McCoy, The (1993)',
 'Wild Card (2015)',
 'Get the Gringo (2012)',
 'Point Blank (1967)',
 'Taken (2008)',
 'Fast & Furious (Fast and the Furious 4, The) (2009)',
 'Unleashed (Danny the Dog) (2005)',
 'Infernal Affairs 2 (Mou gaan dou II) (2003)',
 'Man Apart, A (2003)',
 'Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006)',
 'Clear and Present Danger (1994)',
 'Boondock Saints, The (2000)',
 'Juice (1992)',
 'Killer, The (Die xue shuang xiong) (1989)',
 'Fight Club (1999)',
 'Get Carter (1971)',
 'Killshot (2008)',
 'Last Man Standing (1996)',
 'Max Payne (2008)',
 "Smokin' Aces (2006)",
 'Collateral (2004)',
 'Crossing Guard, The (1995)',
 'Dark Bl

In [121]:
# lets get second recomendation system. on collaborative_filtering
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
ratings = pd.read_csv('ratings.csv')
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [122]:
from tqdm import tqdm
import numpy as np

In [123]:
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

610

In [124]:
movie_vector = {}

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)
    
    for i in range(len(group.userId.values)):
        u = group.userId.values[i]
        r = group.rating.values[i]
        movie_vector[movie][int(u - 1)] = r

100%|██████████| 9719/9719 [00:02<00:00, 3736.49it/s]


In [97]:
movie_vector['Fight Club (1999)']

array([5. , 0. , 0. , 2. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. ,
       0. , 2.5, 3.5, 4.5, 4.5, 5. , 0. , 2. , 3.5, 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 0. , 0. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. ,
       0. , 4. , 3. , 0. , 0. , 5. , 0. , 2. , 0. , 0. , 3. , 0. , 4.5,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5. , 5. , 4.5, 4.5,
       5. , 4.5, 2.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4.5, 0. , 0. ,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 3.5, 0. , 0. , 0. , 0. , 4.5,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 5. , 0. ,
       5. , 0. , 0. , 0. , 0. , 4.5, 5. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 5. , 4.5, 0. , 5. , 0. , 0. , 0. , 4. , 0. ,
       0. , 4. , 0. , 0. , 5. , 0. , 3. , 0. , 0. , 0. , 5. , 0. , 0. ,
       4. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5. , 0.5, 0. , 4. , 0. ,
       5. , 0. , 3. , 4.5, 0. , 0. , 0. , 0. , 0. , 4.5, 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 4. , 4.5, 0. , 4.5, 0.

In [99]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [100]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    if key == my_fav_film:
        continue
    
    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

len(distances)

100%|██████████| 9719/9719 [00:00<00:00, 25037.68it/s]


9718

In [102]:
distances[:100]

[0.9223029947150825,
 0.9300726952435743,
 0.950553928616633,
 1.0,
 1.0,
 0.968921197886033,
 0.877349727371666,
 0.9378423957720661,
 0.7578680102442159,
 0.9065861273541315,
 1.0,
 0.9221663681115744,
 0.9300726952435743,
 1.0,
 0.9688416105544041,
 0.9321605497351766,
 0.8177603774126567,
 0.9006284558932834,
 0.6966660945647201,
 0.9378423957720661,
 0.8554149105432228,
 0.9377944183183576,
 0.9378423957720661,
 0.8308810457311764,
 0.7940122610393006,
 1.0,
 0.9223029947150825,
 0.9102832261670503,
 1.0,
 1.0,
 1.0,
 0.9011078572332661,
 0.8898839486247883,
 1.0,
 0.6597253123703614,
 0.9378423957720661,
 0.9223029947150825,
 0.9223029947150825,
 0.9013115487837553,
 0.7782107904535396,
 0.7514006463315334,
 0.8961467827092279,
 0.9729443134954915,
 0.7861980561672296,
 0.9102832261670503,
 0.9223029947150825,
 0.9223029947150825,
 0.9615419444796035,
 0.7710626622456545,
 0.7265224651196707]

In [125]:
best_indexes = np.argsort(distances)[:100]
best_indexes

array([5511, 5571,  420, 4736, 5203,  405, 6864, 5204, 4322, 4737, 7730,
       7592, 2791, 7078, 5205, 2334, 3498, 3473, 7522, 2163, 6645, 3554,
       3563, 3157, 7390, 2488,  837, 9118, 8884, 1839, 7791,  400, 7703,
       9119, 8001, 8000, 7072, 7679, 1624, 8953, 1048, 5779, 5676, 3032,
       5303, 6769, 6235, 4359, 7420, 7590, 3499, 7652, 7624, 3633, 2940,
        946, 2997, 5775, 6254, 4333, 6943, 1292, 7920, 1803, 6310, 3655,
       7409,  900, 6440, 8002, 3280, 1601, 1606, 3008, 2464, 5509, 2394,
       9214,  427, 4343, 6221, 7997, 8643,  744, 4344,  435, 1146, 1294,
        327, 2059, 6807, 4043, 5581, 8366, 1295, 2681, 4441, 9614, 9105,
       3427], dtype=int64)

In [126]:
movies_rec_syst_2 = [titles[i] for i in best_indexes]
movies_rec_syst_2

['Matrix, The (1999)',
 'Memento (2000)',
 'American History X (1998)',
 'Kill Bill: Vol. 1 (2003)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'American Beauty (1999)',
 'Pulp Fiction (1994)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Inception (2010)',
 'Kill Bill: Vol. 2 (2004)',
 'Sixth Sense, The (1999)',
 'Shawshank Redemption, The (1994)',
 'Eternal Sunshine of the Spotless Mind (2004)',
 'Reservoir Dogs (1992)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Departed, The (2006)',
 'Godfather, The (1972)',
 'Gladiator (2000)',
 'Seven (a.k.a. Se7en) (1995)',
 'Dark Knight, The (2008)',
 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
 'Good Will Hunting (1997)',
 'Goodfellas (1990)',
 'Forrest Gump (1994)',
 'Saving Private Ryan (1998)',
 'Donnie Darko (2001)',
 'Batman Begins (2005)',
 'Usual Suspects, The (1995)',
 'Trainspotting (1996)',
 'Clockwork Orange, A (1971)',
 'Snatch (2000)',
 "Amelie (Fabuleux destin d'Am

In [ ]:
# We have two recomendation systems.
#lets get weight of each film in every system by 0.5. 
#Then we will get two dictionaries of both system and we will merge them. Wich value of system wil==1? these films we will recomend

In [129]:
dict_1 = {}
dict_2 = {}

for film in movies_rec_syst_1:
    dict_1[film] =0.5
    
for film in movies_rec_syst_2:
    dict_2[film] =0.5

In [136]:
from collections import defaultdict

def merge_dicts(dict1, dict2):
    result = defaultdict(int)
    for d in (dict1, dict2):
        for key, value in d.items():
            result[key] += value
    return dict(result)
 
full_recomend_movies = merge_dicts(dict_1, dict_2)   
full_recomend_movies

{'Double Jeopardy (1999)': 0.5,
 'Ninja (2009)': 0.5,
 'Patriot Games (1992)': 0.5,
 'Kidnapping Mr. Heineken (2015)': 0.5,
 'The Fate of the Furious (2017)': 0.5,
 'I Am Wrath (2016)': 0.5,
 'Run All Night (2015)': 0.5,
 'Munich (2005)': 0.5,
 'Death Wish (2018)': 0.5,
 'Baader Meinhof Komplex, Der (2008)': 0.5,
 'Contraband (2012)': 0.5,
 'Shot Caller (2017)': 0.5,
 'Real McCoy, The (1993)': 0.5,
 'Wild Card (2015)': 0.5,
 'Get the Gringo (2012)': 0.5,
 'Point Blank (1967)': 0.5,
 'Taken (2008)': 0.5,
 'Fast & Furious (Fast and the Furious 4, The) (2009)': 0.5,
 'Unleashed (Danny the Dog) (2005)': 0.5,
 'Infernal Affairs 2 (Mou gaan dou II) (2003)': 0.5,
 'Man Apart, A (2003)': 0.5,
 'Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006)': 0.5,
 'Clear and Present Danger (1994)': 0.5,
 'Boondock Saints, The (2000)': 0.5,
 'Juice (1992)': 0.5,
 'Killer, The (Die xue shuang xiong) (1989)': 0.5,
 'Fight Club (1999)': 0.5,
 'Get Carter (1971)': 0.5,
 'Killshot (2008

In [137]:
recomended_films =[]

for k, v in full_recomend_movies.items():
    if v==1:
        recomended_films.append(k)
        
recomended_films# these films we will recomend for "fight_club"

['Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
 'Die Hard (1988)',
 'Bourne Ultimatum, The (2007)',
 'Kill Bill: Vol. 1 (2003)']